In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../data/raw/44V-GEU314__Informe_Completo_2022-05-25_12-16-58_Seguimiento del sensor.csv', encoding='utf-8', sep=';', index_col=0)
df.head(10)

,Fecha y Hora,Coordenadas,Localización,Aceleracion Frenadas*,Altitud*,Bateria Respaldo*,Bateria Vehiculo*,Curso*,eng_eff_on_fuelconsmath*,Ignicion*,Kilometraje*,Velocidad*,Tarjeta*,Tarjetas*,Velocidad,Conductor,Unnamed: 17
№,,,,,,,,,,,,,,,,,
1,2022-05-01 00:36:30,"4.604121, -74.145759","Kennedy, Colombia, Bogotá, Transversal 72Fbis ...",----,2599.90 mt,96.00 %,12.77 V,139.00 º,----,Apagado,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
2,2022-05-01 01:35:58,"4.604121, -74.145759","Kennedy, Colombia, Bogotá, Transversal 72Fbis ...",----,2599.90 mt,96.00 %,12.77 V,139.00 º,1.00,Apagado,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
3,2022-05-01 02:00:06,"4.604121, -74.145759","Kennedy, Colombia, Bogotá, Transversal 72Fbis ...",----,2599.90 mt,----,----,139.00 º,1.00,----,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
4,2022-05-01 02:00:30,"4.603913, -74.145905","Kennedy, Colombia, Bogotá, Carrera 72G 42B S 68",----,2568.30 mt,----,----,0.00 º,----,----,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
5,2022-05-01 02:00:30,"4.603913, -74.145905","Kennedy, Colombia, Bogotá, Carrera 72G 42B S 68",----,2568.30 mt,96.00 %,0.00 V,0.00 º,1.00,Apagado,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
6,2022-05-01 02:58:05,"4.603913, -74.145905","Kennedy, Colombia, Bogotá, Carrera 72G 42B S 68",----,2568.30 mt,96.00 %,12.77 V,0.00 º,1.00,Apagado,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
7,2022-05-01 03:57:38,"4.603913, -74.145905","Kennedy, Colombia, Bogotá, Carrera 72G 42B S 68",----,2568.30 mt,96.00 %,12.77 V,0.00 º,1.00,Apagado,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
8,2022-05-01 04:57:37,"4.603913, -74.145905","Kennedy, Colombia, Bogotá, Carrera 72G 42B S 68",----,2568.30 mt,96.00 %,12.76 V,0.00 º,1.00,Apagado,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN
9,2022-05-01 05:57:39,"4.603913, -74.145905","Kennedy, Colombia, Bogotá, Carrera 72G 42B S 68",----,2568.30 mt,96.00 %,12.76 V,0.00 º,1.00,Apagado,59656.20 km,0.00 Km/h,----,----,0 km/h,----,NaN


In [4]:
for column in df.select_dtypes('object').columns:
    df[column] = df[column].replace(r'----', np.NaN, regex=True)

In [15]:
df.iloc[:,16].value_counts()

Series([], Name: Unnamed: 17, dtype: int64)

In [71]:
df_rep = pd.DataFrame()
df_rep['datetime'] = pd.to_datetime(df['Fecha y Hora'], format='%Y-%m-%d %H:%M:%S')
df_rep['latitude'] = df['Coordenadas'].str.split(',').str[0]
df_rep['longitude'] = df['Coordenadas'].str.split(',').str[1]
df_rep['localization'] = df['Localización']
df_rep['altitude'] = df['Altitud*'].str.extract(r'(\d*.?\d{0,2})', expand=False)
df_rep['battery_perc'] = df['Bateria Respaldo*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')/100
df_rep['battery_volt'] = df['Bateria Vehiculo**'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')
df_rep['course'] = df['Curso'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')
df_rep['eng_eff'] = df['eng_eff_on_fuelconsmath*'].astype('float32')
df_rep['ignition'] = df['Ignición'].apply(lambda x: True if x == 'Encendido' else False)
df_rep['millage'] = df['Kilometraje*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')
df_rep['velocity'] = df['Velocidad*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')

In [18]:
(df['Velocidad*'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32') == df['Velocidad'].str.extract(r'(\d*.?\d{0,2})', expand=False).astype('float32')).value_counts()

True     8165
False      16
dtype: int64

In [72]:
df_rep.sample(10)

,datetime,latitude,longitude,localization,altitude,battery_perc
№,,,,,,
3171,2022-05-11 06:13:26,4.640907,-74.114536,"Puente Aranda, Colombia, Bogotá, Avenida Carre...",2571.40,0.93
3639,2022-05-12 06:24:46,4.692304,-74.120888,"Engativá, Colombia, Bogotá, Avenida Calle 63 96",2569.60,0.92
2532,2022-05-09 06:22:28,4.696827,-74.091482,"Engativá, Colombia, Bogotá, Avenida Calle 80 7...",2568.70,0.93
5331,2022-05-17 07:19:49,4.712555,-74.097253,"Engativá, Colombia, Bogotá, Avenida Calle 90 9...",2572.60,0.91
3150,2022-05-11 05:52:29,4.605274,-74.131223,"Puente Aranda, Colombia, Bogotá, Avenida Carre...",2578.50,0.93
3525,2022-05-11 17:10:28,4.615153,-74.120775,"Puente Aranda, Colombia, Bogotá, Calle 2 53D 67",2580.90,0.92
5102,2022-05-16 12:43:34,4.723828,-74.089704,"Suba, Colombia, Bogotá, Calle 129 91 29",2580.90,0.91
4301,2022-05-13 15:14:38,4.742439,-74.029393,"Usaquén, Colombia, Bogotá, Calle 165 21A 74",2592.80,0.92
4339,2022-05-13 16:03:29,4.714794,-74.032715,"Usaquén, Colombia, Bogotá, Avenida Carrera 9 134A",2581.80,0.92


In [82]:
df['Ignicion*'].value_counts()

Encendido    7169
Apagado       884
Name: Ignicion*, dtype: int64

In [81]:
pd.Series([1, 2, None, 0]).astype('bool')

0     True
1     True
2     True
3    False
dtype: bool